## Data loader

In [2]:
## imports
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

In [27]:
class GPTDatasetV1(Dataset):

    def __init__(self, text, tokenizer,max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special = {'<|endoftext|>'})
        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i: i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))


    def __len__(self):
        return len(self.input_ids)
    

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [28]:
def create_dataloader(text, batch_size, max_length, stride, shuffle = True, drop_last =True, num_workers = 0):

    tokenizer = tiktoken.get_encoding('gpt2')

    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

In [29]:
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

In [30]:
vocab_size = 50257
output_dim = 256 #embdding dimention
context_length = 1024 #block size


In [31]:
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [32]:
batch_size = 8
max_length = 4

In [33]:
dataloader = create_dataloader(raw_text,batch_size, max_length=max_length, stride=max_length)

In [35]:
for batch in dataloader:
    x, y = batch

    token_embeddings = token_embedding_layer(x)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))

    input_embeddings = token_embeddings+pos_embeddings

    break    

In [41]:
print(input_embeddings.shape)

torch.Size([8, 4, 256])
